<table>
    <tr>
        <td><img src="https://s3.amazonaws.com/media-p.slid.es/uploads/1485763/images/9060062/Header.png" width="300"/></td>
        <td>&nbsp;</td>
        <td>
            <h1 style="font-size:200%;color:blue;text-align:center">    <FONT COLOR="blue">  Estudio de Caso 2  </FONT>         </h1></td>         
        <td>
            <tp><p style="font-size:99%;text-align:center">PLN </p></tp>
            <tp><p style="font-size:115%;text-align:center">Pregrado MACC 2023-2</p></tp>
            <tp><p style="font-size:115%;text-align:center">Prof. Fabián Sánchez</p></tp>
        </td>
    </tr>
</table>

<FONT SIZE=5 COLOR="red"> Laura Gonzalez y Dafne Castellanos

# <FONT SIZE=5 COLOR="green"> **Objetivo del estudio de caso y resultados de aprendizaje** </FONT>

En este estudio de caso se espera que el estudiante aplique los conceptos de ***Topic Modeling*** a un conjunto de datos de *Amazon_Reviews* en español.

- Cargar y explorar datos estructurados que incluyen cadenas de texto.

- Aplicar las técnicas de limpieza para normalizar un texto.

- Reconocer los elementos de la creación de una bolsa de palabras por vectorización con frecuencias y con **TF-iDF**.

- Aplicar el método *LSI* y *LDA* para hacer topic modeling con textos

# <FONT SIZE=5 COLOR="green"> **Indicaciones** </FONT>

- Para entregar en grupos de máximo tres integrantes.

- Cargar en la plataforma *e-aulas* los archivos: *.ipynb* y *.pdf*

- Fecha de entrega el día **miércoles 11 de octubre de 2023 antes de las 6:00 p.m.**





In [45]:
!pip install -q stop-words pyLDAvis pandas==1.5.1

In [46]:
!pip install stop-words

In [47]:
!python -m spacy download es_core_news_md

2023-10-14 01:38:46.225517: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 MB 8.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_md')


In [48]:
import pandas as pd

# Para graficar
import plotly.express as px
import pyLDAvis
import pyLDAvis.gensim

# Procesamiento de texto
import re
import nltk
from stop_words import get_stop_words
# Para lematización
import spacy

# Modelos con gensim
from gensim            import  corpora
from gensim.models     import  LsiModel, LdaModel, CoherenceModel, Word2Vec, TfidfModel
from gensim.matutils   import Sparse2Corpus
import gensim

# Para controlar los warnings
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=ImportWarning)

In [49]:
nltk.download("all")

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package basque_grammars is already up-to-date!
[nltk_data]    | Downloading package bcp47 to /root/nltk_data...
[nltk_data]    |   Package bcp47 is already up-to-dat

True

# <FONT SIZE=5 COLOR="bronw"> **Punto 1. Carga y Exploración de los datos** </FONT>

Considere la siguiente base donde se reportan comentarios de productos de Amazon. Estos datos están en español.

"https://raw.githubusercontent.com/Fabian830348/Bases_Datos/master/ReviewAmazonEsp.json"

1. Cargue los datos y haga una breve descripción de estos.

2. Haga un gráfico de barras de la variable *product_category*

3. Haga un gráfico de tortas para la variable "*stars*".

4. Haga una tabla que indique cuántas reseñas por cada categoría de producto hay en el conjunto de datos. Gráfique.

5. ¿Las reseñas (*reviews*) del conjunto de datos a cuántos productos corresponden?





## 1. Cargue los datos y haga una breve descripción de estos.

In [50]:
# Cargue los datos y haga una breve descripción de estos.
datos = pd.read_json('https://raw.githubusercontent.com/Fabian830348/Bases_Datos/master/ReviewAmazonEsp.json', lines = True)
datos.head()

,review_id,product_id,reviewer_id,stars,review_body,review_title,language,product_category
0,es_0417480,product_es_0873923,reviewer_es_0672978,1,"Malisimo, muy grande demasiado aparatoso y mal...",Mala compra,es,wireless
1,es_0180432,product_es_0713146,reviewer_es_0100858,1,No he recibido el pedido no la devolución,No lo he recibido,es,apparel
2,es_0144850,product_es_0356874,reviewer_es_0486447,1,"Tengo que buscar otro sistema, este no funcion...",Que no aprieta bien en el manillar,es,sports
3,es_0339629,product_es_0939832,reviewer_es_0894703,1,Utilicé las brocas de menor diámetro y se me d...,Brocas de mantequilla,es,home_improvement
4,es_0858362,product_es_0489066,reviewer_es_0887663,1,No me gusta su olor a viejo y aspecto malo,No me gusta,es,beauty


In [51]:
datos.shape

(5000, 8)

La base cuenta con 8 columnas y 5000 registros.
Las columnas contienen la siguiente información:

1. **review_id**: Este sería un identificador único o número de referencia para cada reseña. Se utiliza para rastrear y diferenciar cada reseña individual en el conjunto de datos.

2. **product_id**: Esta columna contiene un identificador único o número de referencia para cada producto que se ha revisado. Permite asociar cada reseña con un producto específico.

3. **reviewer_id**: Aquí se almacena un identificador único o número de referencia para cada revisor o usuario que ha escrito una reseña.

4. **stars**: Esta columna indica la calificación o puntuación otorgada por el revisor al producto. Por lo general, se usa un sistema de estrellas, donde un número más alto de estrellas indica una calificación más positiva.

5. **review_body**: En esta columna se encuentra el contenido principal de la reseña escrita por el revisor. Contiene la opinión, comentarios y detalles sobre la experiencia del revisor con el producto.

6. **review_title**: Aquí se almacena el título o encabezado de la reseña, que generalmente resume el contenido de la reseña o destaca un aspecto clave de la experiencia del revisor con el producto.

7. **language**: Esta columna podría contener información sobre el idioma en el que se escribió la reseña. Esto es útil para clasificar y analizar reseñas en diferentes idiomas.

8. **product_category**: Indica la categoría o tipo de producto al que pertenece el producto revisado. Esto es útil para organizar y analizar reseñas en función de las diferentes categorías de productos disponibles en la plataforma.


## 2. Haga un gráfico de barras de la variable product_category

In [52]:
# Calcular el recuento de cada categoría
category_counts = datos['product_category'].value_counts().reset_index()
category_counts.columns = ['product_category', 'count']

# Crear el gráfico de barras con el recuento
fig = px.bar(category_counts, x='product_category', y='count', title = 'Gráfico de barras de la variable "product_category"')

# Mostrar el gráfico
fig.show()

## 3. Haga un gráfico de tortas para la variable "stars".

In [53]:
# Calcular el recuento de cada categoría
stars_counts = datos['stars'].value_counts().reset_index()
stars_counts.columns = ['stars', 'count']

# Crear el gráfico de torta con el recuento
fig = px.pie(stars_counts, names='stars', values='count', title = 'Gráfico de tortas para la variable "stars"')

# Mostrar el gráfico
fig.show()

## 4. Haga una tabla que indique cuántas reseñas por cada categoría de producto hay en el conjunto de datos. Gráfique.



In [54]:
# Calcular el número de reseñas por cada categoría de producto
category_counts = datos['product_category'].value_counts().reset_index()
category_counts.columns = ['product_category', 'count']

# Crear la tabla
print("Tabla de Reseñas por Categoría de Producto:")
print(category_counts)

# Crear el gráfico de barras
fig = px.bar(category_counts, x='product_category', y='count', title="Reseñas por Categoría de Producto")

# Mostrar el gráfico
fig.show()

Tabla de Reseñas por Categoría de Producto:
            product_category  count
0                   wireless    672
1                       home    659
2                        toy    332
3                     sports    331
4           home_improvement    299
5                         pc    273
6                electronics    251
7                     beauty    195
8                 automotive    173
9                    kitchen    156
10                      book    142
11                 drugstore    137
12                 furniture    135
13                   apparel    133
14              baby_product    117
15            office_product    112
16                     other    103
17           lawn_and_garden    101
18                   luggage    101
19              pet_products     89
20  personal_care_appliances     77
21                    camera     64
22                     shoes     60
23    digital_ebook_purchase     51
24                     watch     47
25       musical_ins

## 5. ¿Las reseñas (reviews) del conjunto de datos a cuántos productos corresponden?

In [55]:
# Contar la cantidad única de productos en base a las reseñas
unique_products = datos['product_id'].nunique()

print(f"Las reseñas corresponden a {unique_products} productos.")

Las reseñas corresponden a 4964 productos.


# <FONT SIZE=5 COLOR="bronw"> **Punto 2. Normalización** </FONT>

2.1 Aplique los conceptos de normalización vistos en clase para hacer la limpieza de los datos. Para ello defina funciones que:

- Eliminen las tildes y conserven la ñ.
- Conviertan todo a minúscula.
- Elimine signos especiales: *@* y *#*, etc.
- Elimine todos los dígitos.
- Elimine stop words, etc. Para este conjunto remueva de las stopwords la palabra "no" y agrege otras como: "q", "x" , "amazon" , "mas", etc. Revise si es necesario agregar otras de acuerdo con el contexto del conjunto de datos.


2.2 Elimine palabras que aparecen menos de 20 veces.

2.3 Cree una función de lematización con la librería *Spacy*. **Ayuda**: Completar la siguiente función que deja solamente: sustantivos, adjetivos, adverbios y verbos.

```python
import spacy
import es_core_news_md
# modelo mediano de español
nlp = es_core_news_md.load()
def lematizacion(corpus, tagg=["NOUN", "ADJ", "ADV", "VERB"]):
  .
  .
  .
```


## 1. Aplique los conceptos de normalización vistos en clase para hacer la limpieza de los datos.

In [56]:
datos.shape

(5000, 8)

In [57]:
# Obtener las categorías únicas en la columna "language"
unique_languages = datos['language'].unique()

# Imprimir las categorías únicas
print("Categorías únicas en la columna 'language':")
for lang in unique_languages:
    print(lang)

Categorías únicas en la columna 'language':
es


In [58]:
def cleanPLN(texto, idioma = "es"):
    # Pasar a minusculas
    texto = texto.lower()
    # Remover caracteres especiales y tildes
    texto = re.sub(r'[á]+', 'a', texto)
    texto = re.sub(r'[é]+', 'e', texto)
    texto = re.sub(r'[í]+', 'i', texto)
    texto = re.sub(r'[ó]+', 'o', texto)
    texto = re.sub(r'[úü]+', 'u', texto)
    texto = re.sub(r'[^A-Za-zñÑ\s]+', '', texto)
    texto = texto.replace('@', '').replace('#', '')
    # Tokenizar
    tokens = nltk.word_tokenize(texto)
    # Eliminar las stop.words y Dejar palabras con más de 3 caracteres
    stop_words=get_stop_words(idioma)
    tokens_clean = [palabras for palabras in tokens if (len(palabras)>3 and (palabras not in stop_words) and palabras!='amazon') or palabras == 'no']
    # Regrese la cadena de texto simplificada.
    texto_limpio = ' '.join(tokens_clean)
    return texto_limpio

In [59]:
datos = datos[datos['review_body'] !='']
datos = datos.dropna(subset=['review_body'])

datos['review_body'] = datos['review_body'].astype(str)
datos['review_body'] = datos['review_body'].apply(lambda x: cleanPLN(x,"es"))

In [60]:
datos.head()

,review_id,product_id,reviewer_id,stars,review_body,review_title,language,product_category
0,es_0417480,product_es_0873923,reviewer_es_0672978,1,malisimo grande demasiado aparatoso protector ...,Mala compra,es,wireless
1,es_0180432,product_es_0713146,reviewer_es_0100858,1,no recibido pedido no devolucion,No lo he recibido,es,apparel
2,es_0144850,product_es_0356874,reviewer_es_0486447,1,buscar sistema no funciona bien abrazadera lle...,Que no aprieta bien en el manillar,es,sports
3,es_0339629,product_es_0939832,reviewer_es_0894703,1,utilice brocas menor diametro doblaron varias ...,Brocas de mantequilla,es,home_improvement
4,es_0858362,product_es_0489066,reviewer_es_0887663,1,no gusta olor viejo aspecto malo,No me gusta,es,beauty


## 2. Elimine palabras que aparecen menos de 20 veces.

In [61]:
import pandas as pd
from collections import Counter


# Paso 1: Contar la frecuencia de palabras en toda la columna
word_counts = Counter(" ".join(datos['review_body']).split())

# Paso 2: Filtrar palabras que aparecen al menos 20 veces
palabras_filtradas = [palabra for palabra, frecuencia in word_counts.items() if frecuencia >= 20]

# Paso 3: Reemplazar el contenido de 'review_body' con las palabras filtradas
datos['review_body'] = datos['review_body'].apply(lambda x: ' '.join([palabra for palabra in x.split() if palabra in palabras_filtradas]))

In [62]:
datos.head()

,review_id,product_id,reviewer_id,stars,review_body,review_title,language,product_category
0,es_0417480,product_es_0873923,reviewer_es_0672978,1,grande demasiado protector pantalla,Mala compra,es,wireless
1,es_0180432,product_es_0713146,reviewer_es_0100858,1,no recibido pedido no devolucion,No lo he recibido,es,apparel
2,es_0144850,product_es_0356874,reviewer_es_0486447,1,sistema no funciona bien lleva no bien hacia a...,Que no aprieta bien en el manillar,es,sports
3,es_0339629,product_es_0939832,reviewer_es_0894703,1,varias hacer presion interior peor compra hech...,Brocas de mantequilla,es,home_improvement
4,es_0858362,product_es_0489066,reviewer_es_0887663,1,no gusta olor aspecto malo,No me gusta,es,beauty


In [63]:
datos.shape

(5000, 8)

## 3. Cree una función de lematización con la librería Spacy. Ayuda: Completar la siguiente función que deja solamente: sustantivos, adjetivos, adverbios y verbos.

In [64]:
# Cargar el modelo mediano de español
nlp = spacy.load("es_core_news_md")

# Definir la función de lematizacion
def lematizacion(corpus, tagg=["NOUN", "ADJ", "ADV", "VERB"]):
  doc = nlp(corpus)
  # Filtrar palabras según las etiquetas gramaticales especificadas
  lemmas_text = [token.lemma_ for token in doc if token.pos_ in tagg]
  return lemmas_text

In [65]:
datos['review_body'] = datos['review_body'].apply(lambda x: lematizacion(x))

In [66]:
datos.head()

,review_id,product_id,reviewer_id,stars,review_body,review_title,language,product_category
0,es_0417480,product_es_0873923,reviewer_es_0672978,1,"[grande, demasiado, protector, pantallar]",Mala compra,es,wireless
1,es_0180432,product_es_0713146,reviewer_es_0100858,1,"[no, recibido, pedido, no, devolucion]",No lo he recibido,es,apparel
2,es_0144850,product_es_0356874,reviewer_es_0486447,1,"[sistema, no, funcionar, bien, llevar, bien, a...",Que no aprieta bien en el manillar,es,sports
3,es_0339629,product_es_0939832,reviewer_es_0894703,1,"[hacer, presion, interior, peor, compra, hecho...",Brocas de mantequilla,es,home_improvement
4,es_0858362,product_es_0489066,reviewer_es_0887663,1,"[no, gustar, olor, aspecto, malo]",No me gusta,es,beauty


# <FONT SIZE=5 COLOR="bronw"> **Punto 3. Bolsa de Palabras** </FONT>

Utilice la librería Gensim y el método **TF+iDF** para vectorizar (embedding) con el fin de crear el diccionario y el corpus para aplicar los métodos LSI y LDA del siguiente punto

- Filtre los tokens que aparecen en menos de 100 documentos o en más del $97\%$ de los documentos.





In [67]:
# Crear un diccionario a partir de las listas de palabras
dictionary = corpora.Dictionary(datos["review_body"].to_list())
dictionary.compactify()

In [68]:
# Filtar los extremos
dictionary.filter_extremes(no_below=100, no_above=0.97, keep_n=None)
dictionary.compactify()

In [69]:
# Crear la bolsa de palabras
bow_corpus = [dictionary.doc2bow(review_body) for review_body in datos['review_body'].to_list()]

In [70]:
# Vectorizar
tfidf = TfidfModel(bow_corpus)
corpus = tfidf[bow_corpus]

# <FONT SIZE=5 COLOR="bronw"> **Punto 4. Modelos de Topic Modeling** </FONT>

4.1 Aplique el método ***LSI*** con 20 temas.

  - Imprima las ecuaciones que resultan de *lsimodel.show_topics*
  - Haga una tabla así: en las filas top 15 de las palabras y en las colunmas los temas.
  - Revise el número óptimo de temas usando la función de coherencia.
  - Repita el primer y segundo item.
  - Concluya

In [71]:
lsi_model = LsiModel(corpus=corpus, num_topics=20, id2word=dictionary, random_seed = 0)

In [72]:
# Obtener las ecuaciones de los temas
topics_lsi = lsi_model.show_topics(num_topics=20, num_words =15)
for topic, words in topics_lsi:
  print(f'Topic {topic+1}: {words}')

Topic 1: -0.766*"no" + -0.351*"bien" + -0.192*"buen" + -0.183*"calidad" + -0.170*"producto" + -0.151*"precio" + -0.131*"funcionar" + -0.123*"llegar" + -0.101*"hacer" + -0.092*"quedar" + -0.084*"comprar" + -0.078*"bastante" + -0.075*"parecer" + -0.074*"solo" + -0.071*"pequeño"
Topic 2: -0.563*"no" + 0.493*"buen" + 0.418*"calidad" + 0.304*"bien" + 0.302*"precio" + 0.142*"producto" + 0.078*"perfecto" + 0.076*"bastante" + 0.058*"material" + 0.056*"bonito" + 0.055*"quedar" + 0.053*"parecer" + 0.052*"mejor" + 0.052*"malo" + 0.051*"compra"
Topic 3: 0.813*"bien" + -0.379*"buen" + -0.296*"calidad" + -0.199*"no" + -0.158*"precio" + 0.116*"funcionar" + -0.098*"producto" + 0.086*"quedar" + 0.057*"bastante" + 0.034*"solo" + -0.031*"material" + -0.029*"malo" + -0.026*"comprar" + 0.025*"venir" + -0.024*"mejor"
Topic 4: 0.659*"llegar" + 0.538*"producto" + -0.219*"calidad" + -0.209*"precio" + -0.172*"no" + -0.148*"bien" + 0.145*"perfecto" + 0.133*"tiempo" + 0.114*"llego" + -0.099*"buen" + 0.088*"espera

In [73]:
# Inicializa una lista para almacenar las 15 palabras principales de cada tópico
top_words_per_topic = []

# Itera a través de los tópicos y extrae las palabras clave
for topic, words in topics_lsi:
    # Divide las palabras clave por el carácter "+" y obtén las primeras 15 palabras
    top_words = [word.split("*")[1][1:-1] for word in words.split(" + ")][:15]
    top_words_per_topic.append(top_words)

# Crea un DataFrame de pandas con las palabras clave como filas y los temas como columnas
df_top_words = pd.DataFrame(top_words_per_topic).transpose()

# Muestra el DataFrame
df_top_words.head(15)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,no,no,bien,llegar,llegar,funcionar,buen,funcionar,perfecto,hacer,quedar,esperar,esperar,quedar,comprar,cumplir,facil,bonito,facil,mejor
1,bien,buen,buen,producto,producto,pequeño,precio,producto,pequeño,quedar,esperar,calidad,precio,venir,cumplir,comprar,cumplir,bastante,bonito,solo
2,buen,calidad,calidad,calidad,funcionar,bien,funcionar,buen,precio,funcionar,perfecto,precio,pequeño,hacer,solo,bastante,solo,facil,mejor,facil
3,calidad,bien,no,precio,calidad,producto,calidad,llegar,esperar,calidad,pequeño,hacer,cumplir,parecer,venir,precio,venir,mejor,cumplir,cumplir
4,producto,precio,precio,no,precio,hacer,pequeño,pequeño,funcionar,mejor,bonito,bonito,solo,bonito,hacer,solo,parecer,parecer,solo,esperar
5,precio,producto,funcionar,bien,buen,buen,quedar,quedar,llegar,perfecto,hacer,quedar,calidad,esperar,facil,calidad,bastante,tiempo,quedar,tiempo
6,funcionar,perfecto,producto,perfecto,tiempo,perfecto,producto,precio,venir,pequeño,poner,malo,mejor,foto,esperar,funcion,poner,hacer,tiempo,hacer
7,llegar,bastante,quedar,tiempo,pequeño,quedar,perfecto,esperar,demasiado,esperar,solo,perfecto,malo,bastante,pequeño,venir,mejor,bueno,comprar,parecer
8,hacer,material,bastante,llego,llego,solo,bonito,mejor,bonito,funcion,venir,pequeño,venir,calidad,precio,esperar,foto,quedar,venir,comprar
9,quedar,bonito,solo,buen,quedar,llegar,grande,calidad,buen,bastante,comprar,cumplir,hacer,solo,parecer,parecer,funcion,pequeño,precio,precio


In [74]:
# Coherencia LSI en TF-IDF
sizes_lsi = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]
cms =  [CoherenceModel(model= LsiModel(corpus=corpus, num_topics=i, id2word=dictionary, random_seed = 0),
                       texts = datos['review_body'].to_list(),
                       corpus    = corpus,
                       coherence = 'c_v') for i in range(1, len(sizes_lsi)+1)]

coherences_lsi = [cm.get_coherence() for cm in cms]

print("-------------------------------------------------------")

r = [f"Coherencia LSI en TF-IDF con {sz} temas: {c}"
       for sz, c in zip(sizes_lsi, coherences_lsi)]

print(*r, sep="\n")

x = range(1, len(sizes_lsi)+1)

fig = px.line(x=x, y=coherences_lsi, labels={"x": "num_topics", "y": "Coherence score"})
fig.update_traces(name="coherence_values")
fig.update_layout(legend=dict(x=0.7, y=1))  # Ajusta la posición de la leyenda
fig.update_layout(title="Coherencia LSI en TF-IDF vs núm. de temas")
fig.show()

-------------------------------------------------------
Coherencia LSI en TF-IDF con 1 temas: 0.3366761434223084
Coherencia LSI en TF-IDF con 2 temas: 0.35947916135166647
Coherencia LSI en TF-IDF con 3 temas: 0.37367873045871125
Coherencia LSI en TF-IDF con 4 temas: 0.36677371961224603
Coherencia LSI en TF-IDF con 5 temas: 0.37730597538396743
Coherencia LSI en TF-IDF con 6 temas: 0.3606046166833475
Coherencia LSI en TF-IDF con 7 temas: 0.3605589707165616
Coherencia LSI en TF-IDF con 8 temas: 0.3654873179133
Coherencia LSI en TF-IDF con 9 temas: 0.37720235881066394
Coherencia LSI en TF-IDF con 10 temas: 0.37199792041540086
Coherencia LSI en TF-IDF con 11 temas: 0.3729893198085053
Coherencia LSI en TF-IDF con 12 temas: 0.3705960776359715
Coherencia LSI en TF-IDF con 13 temas: 0.3594648321153159
Coherencia LSI en TF-IDF con 14 temas: 0.3530602945371018
Coherencia LSI en TF-IDF con 15 temas: 0.36091224830232077
Coherencia LSI en TF-IDF con 16 temas: 0.36448564233112213
Coherencia LSI en TF

In [75]:
# Encuentra el índice del valor máximo de coherencia
indice_max_coherence_lsi = coherences_lsi.index(max(coherences_lsi))

# Usa el índice para obtener el tamaño asociado a la coherencia máxima
best_size_lsi = sizes_lsi[indice_max_coherence_lsi]
best_coherence_lsi = coherences_lsi[indice_max_coherence_lsi]

# Imprime los resultados
print(f"El mejor tamaño es {best_size_lsi} con una coherencia de {best_coherence_lsi}")

El mejor tamaño es 5 con una coherencia de 0.37730597538396743


In [76]:
best_lsi_model = LsiModel(corpus=corpus, num_topics=best_size_lsi, id2word=dictionary, random_seed = 0)

In [77]:
# Obtener las ecuaciones de los temas
topics_lsi_best = best_lsi_model.show_topics(num_topics=best_size_lsi, num_words = 15)
for topic, words in topics_lsi_best:
  print(f'Topic {topic+1}: {words}')

Topic 1: 0.766*"no" + 0.351*"bien" + 0.192*"buen" + 0.183*"calidad" + 0.170*"producto" + 0.151*"precio" + 0.131*"funcionar" + 0.123*"llegar" + 0.101*"hacer" + 0.092*"quedar" + 0.084*"comprar" + 0.078*"bastante" + 0.075*"parecer" + 0.074*"solo" + 0.071*"pequeño"
Topic 2: -0.563*"no" + 0.493*"buen" + 0.418*"calidad" + 0.304*"bien" + 0.302*"precio" + 0.142*"producto" + 0.078*"perfecto" + 0.076*"bastante" + 0.058*"material" + 0.056*"bonito" + 0.055*"quedar" + 0.053*"parecer" + 0.052*"mejor" + 0.052*"malo" + 0.051*"compra"
Topic 3: -0.813*"bien" + 0.379*"buen" + 0.296*"calidad" + 0.199*"no" + 0.158*"precio" + -0.116*"funcionar" + 0.098*"producto" + -0.086*"quedar" + -0.057*"bastante" + -0.034*"solo" + 0.031*"material" + 0.029*"malo" + 0.026*"comprar" + -0.025*"venir" + 0.024*"mejor"
Topic 4: 0.659*"llegar" + 0.538*"producto" + -0.219*"calidad" + -0.209*"precio" + -0.172*"no" + -0.148*"bien" + 0.145*"perfecto" + 0.133*"tiempo" + 0.114*"llego" + -0.099*"buen" + 0.088*"esperar" + 0.084*"hacer"

In [78]:
# Inicializa una lista para almacenar las 15 palabras principales de cada tópico
top_words_per_topic = []

# Itera a través de los tópicos y extrae las palabras clave
for topic, words in topics_lsi_best:
    # Divide las palabras clave por el carácter "+" y obtén las primeras 15 palabras
    top_words = [word.split("*")[1][1:-1] for word in words.split(" + ")][:15]
    top_words_per_topic.append(top_words)

# Crea un DataFrame de pandas con las palabras clave como filas y los temas como columnas
df_top_words = pd.DataFrame(top_words_per_topic).transpose()

# Muestra el DataFrame
df_top_words.head(15)

,0,1,2,3,4
0,no,no,bien,llegar,llegar
1,bien,buen,buen,producto,producto
2,buen,calidad,calidad,calidad,funcionar
3,calidad,bien,no,precio,calidad
4,producto,precio,precio,no,precio
5,precio,producto,funcionar,bien,buen
6,funcionar,perfecto,producto,perfecto,tiempo
7,llegar,bastante,quedar,tiempo,pequeño
8,hacer,material,bastante,llego,llego
9,quedar,bonito,solo,buen,quedar


4.2 Aplique el método ***LDA*** con 20 temas.

  - Imprima las ecuaciones con *ldamodel.show_topics*
  - Haga una tabla así: en las filas top 15 de las palabras y en las colunmas los temas.
  - Revise el número óptimo de temas usando la función de coherencia.
  - Repita el primer y segundo item.
  - Concluya

In [79]:
lda_model = LdaModel(corpus=corpus, num_topics=20, id2word=dictionary, random_state = 0, passes = 10)

In [80]:
# Obtener las ecuaciones de los temas
topics_lda = lda_model.show_topics(num_topics=20, num_words = 15)
for topic, words in topics_lda:
  print(f'Topic {topic+1}: {words}')

Topic 1: 0.441*"mejor" + 0.378*"bueno" + 0.055*"precio" + 0.038*"no" + 0.027*"producto" + 0.026*"calidad" + 0.025*"bien" + 0.000*"comprar" + 0.000*"hacer" + 0.000*"dejar" + 0.000*"buen" + 0.000*"recomendar" + 0.000*"decir" + 0.000*"poner" + 0.000*"foto"
Topic 2: 0.274*"rapido" + 0.256*"comprar" + 0.162*"momento" + 0.094*"mes" + 0.041*"llegar" + 0.040*"bien" + 0.039*"funcionar" + 0.039*"no" + 0.027*"buen" + 0.019*"hacer" + 0.004*"producto" + 0.000*"primero" + 0.000*"dejar" + 0.000*"perfecto" + 0.000*"bonito"
Topic 3: 0.226*"parecer" + 0.196*"gustado" + 0.192*"verdad" + 0.124*"vez" + 0.065*"no" + 0.043*"bien" + 0.039*"bastante" + 0.033*"hacer" + 0.033*"producto" + 0.022*"precio" + 0.014*"buen" + 0.008*"calidad" + 0.000*"pequeño" + 0.000*"dejar" + 0.000*"primero"
Topic 4: 0.192*"llego" + 0.164*"tener" + 0.162*"funda" + 0.132*"vendedor" + 0.064*"producto" + 0.047*"perfecto" + 0.041*"bonito" + 0.040*"no" + 0.028*"buen" + 0.026*"bien" + 0.020*"llegar" + 0.020*"venir" + 0.018*"parte" + 0.011*

In [81]:
# Inicializa una lista para almacenar las 15 palabras principales de cada tópico
top_words_per_topic = []

# Itera a través de los tópicos y extrae las palabras clave
for topic, words in topics_lda:
    # Divide las palabras clave por el carácter "+" y obtén las primeras 15 palabras
    top_words = [word.split("*")[1][1:-1] for word in words.split(" + ")][:15]
    top_words_per_topic.append(top_words)

# Crea un DataFrame de pandas con las palabras clave como filas y los temas como columnas
df_top_words = pd.DataFrame(top_words_per_topic).transpose()

# Muestra el DataFrame
df_top_words.head(15)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,mejor,rapido,parecer,llego,bien,venir,perfecto,perfectamente,problema,tiempo,material,plastico,foto,esperar,producto,solo,quedar,no,pequeño,calidad
1,bueno,comprar,gustado,tener,menos,genial,facil,comodo,recomendar,compra,buen,caja,grande,no,buen,llevar,llegar,cumplir,funcionar,precio
2,precio,momento,verdad,funda,primero,tamaño,bonito,unico,no,compre,calidad,venir,color,bien,poner,decir,bastante,funcion,color,buen
3,no,mes,vez,vendedor,no,adema,bien,dejar,comprar,demasiado,bastante,parecer,tamaño,llegar,hacer,parte,bien,dinero,duro,gustar
4,producto,llegar,no,producto,precio,bien,buen,malo,bien,llegar,funda,producto,pequeño,producto,no,no,no,hacer,original,semana
5,calidad,bien,bien,perfecto,calidad,buen,no,calidad,funcionar,buen,parecer,bien,bonito,precio,bien,funcionar,producto,producto,no,producto
6,bien,funcionar,bastante,bonito,producto,hacer,producto,funcionar,hacer,bien,hacer,no,parecer,unico,calidad,bien,hacer,bien,bien,bien
7,comprar,no,hacer,no,hacer,no,calidad,buen,primero,precio,original,malo,no,buen,unico,poner,funcion,perfectamente,hacer,no
8,hacer,buen,producto,buen,color,calidad,precio,bien,producto,calidad,no,calidad,bien,pequeño,precio,mes,perfecto,precio,bastante,comprar
9,dejar,hacer,precio,bien,buen,bastante,funcionar,bastante,mes,no,demasiado,bastante,buen,bastante,llegar,hacer,poner,buen,producto,llegar


In [82]:
# Coherencia LDA en TF-IDF
sizes_lda = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]
cms =  [CoherenceModel(model= LdaModel(corpus=corpus, num_topics=i, id2word=dictionary, random_state = 0, passes = 10),
                       texts = datos['review_body'].to_list(),
                       corpus    = corpus,
                       coherence = 'c_v') for i in range(1, len(sizes_lda)+1)]

coherences_lda = [cm.get_coherence() for cm in cms]

print("-------------------------------------------------------")

r = [f"Coherencia LDA en TF-IDF con {sz} temas: {c}"
       for sz, c in zip(sizes_lda, coherences_lda)]

print(*r, sep="\n")

x = range(1, len(sizes_lda)+1)

fig = px.line(x=x, y=coherences_lda, labels={"x": "num_topics", "y": "Coherence score"})
fig.update_traces(name="coherence_values")
fig.update_layout(legend=dict(x=0.7, y=1))  # Ajusta la posición de la leyenda
fig.update_layout(title="Coherencia LDA en TF-IDF vs núm. de temas")
fig.show()

-------------------------------------------------------
Coherencia LDA en TF-IDF con 1 temas: 0.28516620335862497
Coherencia LDA en TF-IDF con 2 temas: 0.3419695819701872
Coherencia LDA en TF-IDF con 3 temas: 0.3671583662437952
Coherencia LDA en TF-IDF con 4 temas: 0.3664564306370858
Coherencia LDA en TF-IDF con 5 temas: 0.3893069260918097
Coherencia LDA en TF-IDF con 6 temas: 0.3727057918416339
Coherencia LDA en TF-IDF con 7 temas: 0.39587557995460226
Coherencia LDA en TF-IDF con 8 temas: 0.3891497688988441
Coherencia LDA en TF-IDF con 9 temas: 0.391366125374427
Coherencia LDA en TF-IDF con 10 temas: 0.3771994572918437
Coherencia LDA en TF-IDF con 11 temas: 0.3821761280986981
Coherencia LDA en TF-IDF con 12 temas: 0.3839134296442315
Coherencia LDA en TF-IDF con 13 temas: 0.3802316657015796
Coherencia LDA en TF-IDF con 14 temas: 0.3905940382917804
Coherencia LDA en TF-IDF con 15 temas: 0.38027476206963906
Coherencia LDA en TF-IDF con 16 temas: 0.38183609724567524
Coherencia LDA en TF-I

In [83]:
# Encuentra el índice del valor máximo de coherencia
indice_max_coherence_lda = coherences_lda.index(max(coherences_lda))

# Usa el índice para obtener el tamaño asociado a la coherencia máxima
best_size_lda = sizes_lda[indice_max_coherence_lda]
best_coherence_lda = coherences_lda[indice_max_coherence_lda]

# Imprime los resultados
print(f"El mejor tamaño es {best_size_lda} con una coherencia de {best_coherence_lda}")

El mejor tamaño es 7 con una coherencia de 0.39587557995460226


In [84]:
best_lda_model = LdaModel(corpus=corpus, num_topics=best_size_lda, id2word=dictionary, random_state = 0, passes = 10)

In [85]:
# Obtener las ecuaciones de los temas
topics_lda_best = best_lda_model.show_topics(num_topics=best_size_lda, num_words = 15)
for topic, words in topics_lda_best:
  print(f'Topic {topic+1}: {words}')

Topic 1: 0.231*"no" + 0.107*"bueno" + 0.092*"comodo" + 0.075*"recomendar" + 0.072*"bastante" + 0.062*"foto" + 0.061*"demasiado" + 0.060*"duro" + 0.057*"hacer" + 0.050*"unico" + 0.049*"poner" + 0.042*"bien" + 0.018*"grande" + 0.011*"llevar" + 0.004*"producto"
Topic 2: 0.139*"funcionar" + 0.132*"pequeño" + 0.115*"comprar" + 0.111*"rapido" + 0.099*"compra" + 0.066*"momento" + 0.054*"mes" + 0.049*"bien" + 0.046*"no" + 0.033*"problema" + 0.026*"hacer" + 0.022*"buen" + 0.017*"llevar" + 0.015*"compre" + 0.014*"bastante"
Topic 3: 0.101*"quedar" + 0.084*"parecer" + 0.069*"color" + 0.064*"material" + 0.057*"gustado" + 0.055*"verdad" + 0.055*"problema" + 0.046*"llevar" + 0.045*"no" + 0.043*"bien" + 0.041*"parte" + 0.038*"primero" + 0.037*"malo" + 0.035*"calidad" + 0.033*"buen"
Topic 4: 0.242*"perfecto" + 0.177*"producto" + 0.116*"esperar" + 0.082*"llego" + 0.076*"decir" + 0.059*"vendedor" + 0.055*"caja" + 0.049*"buen" + 0.037*"no" + 0.029*"bien" + 0.026*"venir" + 0.015*"dinero" + 0.011*"poner" + 

In [91]:
# Inicializa una lista para almacenar las 15 palabras principales de cada tópico
top_words_per_topic = []

# Itera a través de los tópicos y extrae las palabras clave
for topic, words in topics_lda_best:
    # Divide las palabras clave por el carácter "+" y obtén las primeras 15 palabras
    top_words = [word.split("*")[1][1:-1] for word in words.split(" + ")][:15]
    top_words_per_topic.append(top_words)

# Crea un DataFrame de pandas con las palabras clave como filas y los temas como columnas
df_top_words = pd.DataFrame(top_words_per_topic).transpose()

# Muestra el DataFrame
df_top_words.head(15)

,0,1,2,3,4,5,6
0,no,funcionar,quedar,perfecto,buen,llegar,cumplir
1,bueno,pequeño,parecer,producto,calidad,facil,bonito
2,comodo,comprar,color,esperar,precio,tiempo,perfectamente
3,recomendar,rapido,material,llego,bien,venir,solo
4,bastante,compra,gustado,decir,mejor,genial,funda
5,foto,momento,verdad,vendedor,tener,gustar,funcion
6,demasiado,mes,problema,caja,no,tamaño,dejar
7,duro,bien,llevar,buen,plastico,bien,grande
8,hacer,no,no,no,semana,adema,original
9,unico,problema,bien,bien,dinero,no,vez


# <FONT SIZE=5 COLOR="bronw"> **Punto 5. Análisis del Modelo LDA y Visualización** </FONT>

De acuerdo con los resultados obtenidos en el punto 4.

- Liste los 5 primeros temas y asígneles una etiqueta (nombre). Explique.

- Realice la visualización interactiva de *LDA*

In [95]:
# Suponiendo que df_top_words tiene más de 5 columnas
df_top_words = df_top_words.iloc[:, :5]

# CAMBIAAAARRR
# Renombra las columnas del DataFrame
df_top_words.columns = ['Satisfacción del Cliente y Comodidad del Producto', 'Experiencia de Compra y Satisfacción del Cliente ',
                        'Apariencia y Satisfacción con la Calidad del Producto', 'Satisfacción con el Producto y la Entrega',
                        'Calidad, Precio y Satisfacción del Cliente']

# Muestra el DataFrame con las etiquetas asignadas
df_top_words.head(15)

,Satisfacción del Cliente y Comodidad del Producto,Experiencia de Compra y Satisfacción del Cliente,Apariencia y Satisfacción con la Calidad del Producto,Satisfacción con el Producto y la Entrega,"Calidad, Precio y Satisfacción del Cliente"
0,no,funcionar,quedar,perfecto,buen
1,bueno,pequeño,parecer,producto,calidad
2,comodo,comprar,color,esperar,precio
3,recomendar,rapido,material,llego,bien
4,bastante,compra,gustado,decir,mejor
5,foto,momento,verdad,vendedor,tener
6,demasiado,mes,problema,caja,no
7,duro,bien,llevar,buen,plastico
8,hacer,no,no,no,semana
9,unico,problema,bien,bien,dinero


1. **Tópico 1 - "Satisfacción del Cliente y Comodidad del Producto":**

Este tópico se centra en la satisfacción del cliente y la comodidad de los productos. Las palabras clave más importantes incluyen "no", "bueno", "comodo", "recomendar", "bastante" y "foto". Los usuarios están compartiendo sus experiencias y opiniones sobre la comodidad, la satisfacción y la recomendación de productos, prestando especial atención a la calidad y la cantidad de productos, así como a su apariencia.

2. **Tópico 2 - "Experiencia de Compra y Satisfacción del Cliente":**

Este tópico se enfoca en la experiencia de compra, la satisfacción del cliente y la velocidad de la transacción. Las palabras clave más importantes incluyen "funcionar", "pequeño", "comprar", "rapido", "compra" y "momento". Los usuarios comparten sus experiencias de compra, destacando la rapidez de las transacciones, la satisfacción con los productos adquiridos y la falta de problemas. Este tópico refleja la importancia de una experiencia de compra positiva y la satisfacción del cliente.

3. **Tópico 3 - "Apariencia y Satisfacción con la Calidad del Producto":**

Este tópico se centra en la apariencia de los productos y la satisfacción del cliente en cuanto a la calidad de los mismos. Las palabras clave más importantes incluyen "quedar", "parecer", "color", "material", "gustado", "verdad", "problema", "llevar", "no" y "bien". Los usuarios discuten la apariencia y las percepciones iniciales de los productos, así como su satisfacción con la calidad. Este tópico refleja la importancia de la apariencia y la calidad de los productos en la experiencia del cliente.

4. **Tópico 4 - "Satisfacción con el Producto y la Entrega":**

Este tópico se centra en la satisfacción de los clientes con el producto y el proceso de entrega. Las palabras clave más importantes incluyen "perfecto", "producto", "esperar", "llego", "decir", "vendedor", "caja", "buen", "no" y "bien". Los usuarios expresan su satisfacción con la calidad del producto, el proceso de entrega, la comunicación del vendedor y otros aspectos relacionados con la compra. Este tópico refleja la importancia de la satisfacción del cliente en relación con el producto y la entrega.

5. **Tópico 5 - "Calidad, Precio y Satisfacción del Cliente":**

Este tópico se enfoca en la calidad del producto, el precio y la satisfacción del cliente. Las palabras clave más importantes incluyen "buen", "calidad", "precio", "bien", "mejor", "dinero", "producto", "malo" y "hacer". Los usuarios discuten la relación entre la calidad y el precio de los productos, expresan sus preferencias por productos de alta calidad, y comentan sobre su satisfacción como clientes. Este tópico refleja la importancia de la calidad y el precio en la satisfacción del cliente.

In [93]:
vis = pyLDAvis.gensim.prepare(topic_model = best_lda_model,
                              corpus = corpus,
                              dictionary = dictionary,
                              R = 10)  # número de palabras claves
pyLDAvis.display(vis)

# <FONT SIZE=5 COLOR="bronw"> **Punto 6. Conclusiones** </FONT>

El análisis exploratorio de datos y el proceso de limpieza y normalización que se realizó en este proyecto proporcionaron una base sólida para llevar a cabo el modelado de tópicos en un conjunto de datos de reseñas de Amazon en español. Los datos contenían reseñas de una amplia variedad de productos agrupados en 30 categorías, con calificaciones que iban desde 1 hasta 5 estrellas. La uniformidad en la distribución de las calificaciones (con 1000 datos por cada calificación) aseguró que el análisis se llevara a cabo en condiciones equitativas.

Para la limpieza de texto, se aplicó una serie de técnicas que incluyeron la eliminación de caracteres especiales, la conversión a minúsculas y la eliminación de palabras poco relevantes. Además, se realizó una lematización para reducir las palabras a sus formas base, lo que facilitó el análisis y mejoró la calidad de los tópicos identificados.

El proceso de modelado de tópicos se dividió en dos enfoques diferentes: Latent Semantic Indexing (LSI) y Latent Dirichlet Allocation (LDA). Cada uno de estos métodos proporciona una perspectiva única sobre la estructura subyacente de las reseñas.

LSI, o Indexación Semántica Latente, es una técnica que busca encontrar relaciones semánticas entre palabras y documentos mediante descomposición de valores singulares (SVD). Se ajustó el modelo LSI con diferentes tamaños de tópicos y se determinó que el tamaño óptimo era 5, con una coherencia de 0.3773. Esto sugiere que 5 tópicos capturan de manera efectiva las estructuras latentes en los datos de reseñas de Amazon en español.

Por otro lado, LDA, o Asignación Latente de Dirichlet, es un modelo generativo que asume que cada documento se crea a partir de una mezcla de tópicos y que cada tópico se representa como una distribución de palabras. En este caso, el modelo LDA se ajustó con diferentes números de tópicos y se encontró que el mejor tamaño era 7, con una coherencia de 0.3959. Este valor de coherencia sugiere que 7 tópicos son los más adecuados para describir la estructura de las reseñas en el conjunto de datos.

Finalmente, la visualización de los tópicos se realizó con la ayuda de pyLDAvis, lo que permitió una representación gráfica de los tópicos identificados en el modelo LDA. Esta visualización es valiosa para comprender y explorar la estructura subyacente de los datos y las relaciones entre los tópicos identificados. Particularmente, se pudo observa que:

**Distribución de Tópicos:**

- Tópico 1 y Tópico 6 están relacionados y se encuentran cercanos en la PC1.
- Tópico 3 y Tópico 4 también están relacionados y se encuentran cercanos en la PC1.
- Tópico 7 se encuentra alejado de los otros tópicos y parece ser distinto.
- Tópico 2 se encuentra sobre la PC2.
- Tópico 5 se encuentra debajo del Tópico 2.

**Palabras Clave más Representativas:**

- Tópico 1: "quedar"
- Tópico 2: "buen"
- Tópico 3: "no"
- Tópico 4: "cumplir"
- Tópico 5: "llegar"
- Tópico 6: "funcionar"
- Tópico 7: "perfecto"

**Distribución de Tokens:**

- Cada tópico contiene aproximadamente entre el 12% y el 17% de los tokens en el conjunto de datos.

Estos hallazgos te proporcionan información valiosa sobre la estructura y la relación entre los tópicos identificados. La cercanía en la proyección de los tópicos en las componentes principales (PC1 y PC2) sugiere similitudes temáticas, mientras que la separación indica diferencias significativas.

Por ejemplo, los Tópicos 1 y 6 parecen estar relacionados y pueden estar discutiendo temas similares relacionados con la satisfacción del cliente y la comodidad de los productos. Del mismo modo, los Tópicos 3 y 4 pueden estar relacionados, centrándose en la apariencia y la satisfacción con la calidad de los productos. Por otro lado, el Tópico 7 se encuentra alejado de los demás y parece ser un tema distinto, posiblemente relacionado con una experiencia de compra "perfecta".

La distribución de palabras clave en cada tópico también proporciona una idea clara de los temas discutidos en las reseñas de Amazon en español y cómo se relacionan con la satisfacción del cliente, la calidad del producto y otros factores relevantes.

# <FONT SIZE=5 COLOR="bronw"> **Punto 7. Bibliografía** </FONT>

Gensim: Latent Dirichlet Allocation. (2022). Radimrehurek.com. https://radimrehurek.com/gensim/models/ldamodel.html

Gensim: Latent Semantic Indexing. (2022). Radimrehurek.com. https://radimrehurek.com/gensim/models/lsimodel.html

Lemmatizer · spaCy API Documentation. (2016). Lemmatizer. https://spacy.io/api/lemmatizer

‌